This code builds the reporting DB based on queries against GBQ. Unlike the other tasks, this one I'll do in a bit more "industrial strength" style. That'll mean more functions and a more modular feel to the code. 

## 1. Connect to GBQ

In [ ]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
import sqlite3

from wedge_functions import *

In [ ]:
# These first two values will be different on your machine. 
service_path = "C:\\users\\jchan\\dropbox\\teaching\\"
service_file = 'UMT-MSBA-7b4265df0ca4.json' # this is your authentication information  
gbq_proj_id = 'umt-msba'  # change this to your project_id
gbq_dataset_id = 'wedge_transactions' # and change this to your data set ID

credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## 2. Build the Destination DB 

In [ ]:
db_name = "wedge_reporting.db"

db = sqlite3.connect(db_name) 
cur = db.cursor()

create_table(cur,"date_hour")
create_table(cur,"owner_year_month")
create_table(cur,"product_year_month")


## 3. Run GBQ Queries, fill tables.

In [ ]:
tables_to_fill = ['date_hour','owner_year_month','product_year_month']

for tbl in tables_to_fill :
    query = get_gbq_query(tbl) 
    
    query_job = client.query(
        query,
        location="US",
    )
    
    query_job.result()
    
    fill_db_table(cur,tbl,query_job)
    db.commit()


In [ ]:
db.close()

In [ ]:
print("Done")